In [1]:
import torch    
torch.manual_seed(123)
import random
random.seed(123)

import torch.nn as nn
import os
import shutil
import itertools

from chofer_torchex.utils.data.collate import dict_sample_target_iter_concat
from chofer_torchex.utils.functional import collection_cascade, cuda_cascade
from chofer_tda_datasets import Reddit5kJmlr
from chofer_tda_datasets.transforms import Hdf5GroupToDict
from jmlr_2018_code.utils import *
from sklearn.model_selection import ShuffleSplit

from chofer_torchex.nn.slayer import SLayerRationalHat, LinearRationalStretchedBirthLifeTimeCoordinateTransform, prepare_batch


from collections import Counter, defaultdict

%matplotlib notebook



from torch.utils.data import DataLoader

os.environ['CUDA_VISIBLE_DEVICES'] = str(0)

class RedditCollate:   
    def __init__(self, dataset, cuda=True):
        self.cuda = cuda
        self.dataset = dataset
        
    def __call__(self, sample_target_iter):
        x, y = dict_sample_target_iter_concat(sample_target_iter)
        
        for k in self.dataset.keys_not_essential:
            batch_view = x[k]
            x[k] = prepare_batch(batch_view, 2)
            
        for k in self.dataset.keys_essential:
            batch_view = x[k]
            x[k] = prepare_batch(batch_view, 1)            

        y = torch.LongTensor(y)    

        if self.cuda:
            # Shifting the necessary parts of the prepared batch to the cuda
            x = {k: collection_cascade(v,
                                       lambda x: isinstance(x, tuple),
                                       lambda x: (x[0].cuda(), x[1].cuda(), x[2], x[3]))
                 for k, v in x.items()}

            y = y.cuda()

        return x, y
    

class train_env:
    n_epochs = 300
    lr_initial = 0.5
    lr_epoch_step = 20
    batch_size = 100
    train_size = 0.9
    nu = 0.01
    momentum = 0.9


/scratch2/chofer/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dataset = Reddit5kJmlr(data_root_folder_path='/scratch1/chofer/jmlr2018_data/')
dataset.keys_essential = ('dim_0_ess', 'dim_1_ess')
dataset.keys_not_essential = ('dim_0',)
dataset.keys_of_interrest = dataset.keys_essential + dataset.keys_not_essential

In [3]:
def unsqueeze_essentials(x):
    for k in dataset.keys_essential:
        if len(x[k]) > 0:
            x[k] = x[k].unsqueeze(1)
            
    return x

def coordinate_transform(x):
    t = LinearRationalStretchedBirthLifeTimeCoordinateTransform(nu=train_env.nu)
    for k in dataset.keys_not_essential:
        x[k] = t(x[k])   
        
    return x

dataset.data_transforms = \
[
    Hdf5GroupToDict(),
    numpy_to_torch_cascade,
    coordinate_transform, 
    unsqueeze_essentials
]
dataset.target_transforms = [lambda x: int(x) - 1]

In [4]:
reddit_collate = RedditCollate(dataset, cuda=False)         

In [5]:
def LinearCell(n_in, n_out):
    m = nn.Sequential(nn.Linear(n_in, n_out), 
                      nn.BatchNorm1d(n_out), 
                      nn.ReLU()
                     )
    m.out_features = m[0].out_features
    return m


def Slayer(n_elements, point_dim):
    return SLayerRationalHat(n_elements, point_dimension=2, radius_init=500)   


class Reddit5KModel(nn.Module):    
    def __init__(self):
        super().__init__()  
        
        dim_0_n_elements = 150
        dim_0_ess_n_elements = 5
        dim_1_ess_n_elements = 50
        
        self.dim_0 = nn.Sequential(
            Slayer(dim_0_n_elements, 2),
            nn.Tanh()
        )
        self.dim_0_ess = nn.Sequential(            
            Slayer(dim_0_ess_n_elements, 1),
            nn.Tanh()
        )
        self.dim_1_ess = nn.Sequential(            
            Slayer(dim_1_ess_n_elements, 1),
            nn.Tanh()
        )
        

        cls_in_size = dim_0_n_elements + dim_0_ess_n_elements + dim_1_ess_n_elements
        n_1 = 2*cls_in_size#int(cls_in_size/2)
#         cls_in_size = self.dim_0_linear.out_features + self.dim_0_ess_linear.out_features + self.dim_1_ess_linear.out_features
        self.cls = nn.Sequential(          
                               nn.Dropout(0.2),
                               LinearCell(cls_in_size, n_1),
                               nn.Dropout(0.2),
                               LinearCell(n_1, int(n_1/4)),
                               LinearCell(int(n_1/4), int(n_1/16)),
                               nn.Linear(int(n_1/16), 5))
                         
    def forward(self, x):
        x_dim_0 = self.dim_0(x['dim_0'])        
        
        x_dim_0_ess = self.dim_0_ess(x['dim_0_ess'])   
        
        x_dim_1_ess = self.dim_1_ess(x['dim_1_ess'])       
        
        x = torch.cat([x_dim_0, x_dim_0_ess, x_dim_1_ess], dim=1)
        x = self.cls(x)
            
        return x
        
        
    def centers_init(self, sample_target_iter):
        x, _ = dict_sample_target_iter_concat(sample_target_iter)
        
        def get_centers(samples, n_centers):    
            s = torch.cat(samples, dim=0)
            s = list({tuple(row) for row in s})
            print(len(s))
            
            if len(s) < n_centers:
                while len(s) < n_centers:
                    s += s 
                    
            s = np.array(s)
            
            if s.ndim == 1:
                s = s.reshape(-1, 1)
                
            kmeans = sklearn.cluster.KMeans(n_clusters=n_centers, 
                                            init='k-means++', 
                                            random_state=123, 
                                            n_init=1)                           
            kmeans.fit(s)
            centers = kmeans.cluster_centers_
            centers = torch.from_numpy(centers).float()    
            return centers
        
        for key, slayer in zip(['dim_0', 'dim_0_ess', 'dim_1_ess'], [self.dim_0[0], self.dim_0_ess[0], self.dim_1_ess[0]]):
            samples = x[key]       
            
            centers = get_centers(samples, slayer.centers.size(0))
            slayer.centers.data = centers

In [ ]:
stats_of_runs = []
def experiment():      
    splitter = ShuffleSplit(n_splits=10, 
                            train_size=train_env.train_size, 
                            test_size=1-train_env.train_size, 
                            random_state=123)
    train_test_splits = list(splitter.split(X=dataset.targets, y=dataset.targets))

    train_test_splits = [(train_i.tolist(), test_i.tolist()) for train_i, test_i in train_test_splits]
    
    for run_i, (train_i, test_i) in enumerate(train_test_splits):
        print('')
        print('Run', run_i)
        
        model = Reddit5KModel()
        model.centers_init([dataset[i] for i in train_i])
        model.cuda()

        stats = defaultdict(list)
        stats_of_runs.append(stats)  
        
        stats['model'] = model        
        
        opt=torch.optim.SGD(model.parameters(), lr=train_env.lr_initial, momentum=train_env.momentum)

        for i_epoch in range(1, train_env.n_epochs+1):      

            model.train()

            train_sampler = [i for i in train_i] 
            random.shuffle(train_sampler)    
            
            dl_train = DataLoader(dataset,
                              batch_size=train_env.batch_size, 
                              collate_fn=reddit_collate,
                              sampler=train_sampler,                               
                              num_workers=5,
                              drop_last=True)

            dl_test = DataLoader(dataset,
                                 batch_size=train_env.batch_size, 
                                 collate_fn=reddit_collate, 
                                 sampler=test_i,
                                 num_workers=5)

            epoch_loss = 0

            if i_epoch % train_env.lr_epoch_step == 0:
                adapt_lr(opt, lambda lr: lr*0.5)

            for i_batch, (x, y) in enumerate(dl_train, 1):   
                
                x = {k: collection_cascade(v,
                                       lambda x: isinstance(x, tuple),
                                       lambda x: (x[0].cuda(), x[1].cuda(), x[2], x[3]))
                     for k, v in x.items()}

                y = y.cuda()

                y = torch.autograd.Variable(y)

                def closure():
                    opt.zero_grad()
                    y_hat = model(x)            
                    loss = nn.functional.cross_entropy(y_hat, y)   
                    loss.backward()
                    return loss

                loss = opt.step(closure)

                epoch_loss += float(loss)
                stats['loss_by_batch'].append(float(loss))
                stats['centers'].append(model.dim_0[0].centers.data.cpu().numpy())
                stats['radius'].append(model.dim_0[0].radius.data.cpu().numpy())

                print("Epoch {}/{}, Batch {}/{}".format(i_epoch, train_env.n_epochs, i_batch, len(dl_train)), end="       \r")

            stats['train_loss_by_epoch'].append(epoch_loss/len(dl_train))

            print("\n\r testing...")
            model.eval()    
            true_samples = 0
            seen_samples = 0
            epoch_test_loss = 0
            for i_batch, (x, y) in enumerate(dl_test):
                
                x = {k: collection_cascade(v,
                                       lambda x: isinstance(x, tuple),
                                       lambda x: (x[0].cuda(), x[1].cuda(), x[2], x[3]))
                 for k, v in x.items()}

                y = y.cuda()

                y_hat = model(x)
                epoch_test_loss += float(nn.functional.cross_entropy(y_hat, torch.autograd.Variable(y.cuda())).data)

                y_hat = y_hat.max(dim=1)[1].data.long()

                true_samples += (y_hat == y).sum()
                seen_samples += y.size(0)  

            stats['test_accuracy'].append(true_samples/seen_samples)
            stats['test_loss_by_epoch'].append(epoch_test_loss/len(dl_test))
            print(true_samples/seen_samples)       
        
experiment()        


Run 0
1492
2
712
Epoch 1/300, Batch 44/44       
 testing...
0.432
Epoch 2/300, Batch 44/44       
 testing...
0.414
Epoch 3/300, Batch 44/44       
 testing...
0.494
Epoch 4/300, Batch 44/44       
 testing...
0.424
Epoch 5/300, Batch 44/44       
 testing...
0.446
Epoch 6/300, Batch 44/44       
 testing...
0.482
Epoch 7/300, Batch 44/44       
 testing...
0.512
Epoch 8/300, Batch 44/44       
 testing...
0.506
Epoch 9/300, Batch 44/44       
 testing...
0.476
Epoch 10/300, Batch 44/44       
 testing...
0.518
Epoch 11/300, Batch 44/44       
 testing...
0.47
Epoch 12/300, Batch 44/44       
 testing...
0.472
Epoch 13/300, Batch 44/44       
 testing...
0.514
Epoch 14/300, Batch 44/44       
 testing...
0.512
Epoch 15/300, Batch 44/44       
 testing...
0.504
Epoch 16/300, Batch 44/44       
 testing...
0.482
Epoch 17/300, Batch 44/44       
 testing...
0.506
Epoch 18/300, Batch 44/44       
 testing...
0.478
Epoch 19/300, Batch 44/44       
 testing...
0.514
Epoch 20/300, Batch 44/

0.582
Epoch 161/300, Batch 44/44       
 testing...
0.57
Epoch 162/300, Batch 44/44       
 testing...
0.58
Epoch 163/300, Batch 44/44       
 testing...
0.574
Epoch 164/300, Batch 44/44       
 testing...
0.568
Epoch 165/300, Batch 44/44       
 testing...
0.576
Epoch 166/300, Batch 44/44       
 testing...
0.58
Epoch 167/300, Batch 44/44       
 testing...
0.576
Epoch 168/300, Batch 44/44       
 testing...
0.574
Epoch 169/300, Batch 44/44       
 testing...
0.578
Epoch 170/300, Batch 44/44       
 testing...
0.568
Epoch 171/300, Batch 44/44       
 testing...
0.566
Epoch 172/300, Batch 44/44       
 testing...
0.58
Epoch 173/300, Batch 44/44       
 testing...
0.582
Epoch 174/300, Batch 44/44       
 testing...
0.574
Epoch 175/300, Batch 44/44       
 testing...
0.572
Epoch 176/300, Batch 44/44       
 testing...
0.576
Epoch 177/300, Batch 44/44       
 testing...
0.578
Epoch 178/300, Batch 44/44       
 testing...
0.588
Epoch 179/300, Batch 44/44       
 testing...
0.584
Epoch 180/

Epoch 178/300, Batch 44/44       
 testing...
0.522
Epoch 179/300, Batch 44/44       
 testing...
0.52
Epoch 180/300, Batch 44/44       
 testing...
0.52
Epoch 181/300, Batch 44/44       
 testing...
0.518
Epoch 182/300, Batch 44/44       
 testing...
0.52
Epoch 183/300, Batch 44/44       
 testing...
0.53
Epoch 184/300, Batch 44/44       
 testing...
0.526
Epoch 185/300, Batch 44/44       
 testing...
0.52
Epoch 186/300, Batch 44/44       
 testing...
0.528
Epoch 187/300, Batch 44/44       
 testing...
0.52
Epoch 188/300, Batch 44/44       
 testing...
0.526
Epoch 189/300, Batch 44/44       
 testing...
0.52
Epoch 190/300, Batch 44/44       
 testing...
0.526
Epoch 191/300, Batch 44/44       
 testing...
0.524
Epoch 192/300, Batch 44/44       
 testing...
0.522
Epoch 193/300, Batch 44/44       
 testing...
0.522
Epoch 194/300, Batch 44/44       
 testing...
0.528
Epoch 195/300, Batch 44/44       
 testing...
0.53
Epoch 196/300, Batch 44/44       
 testing...
0.536
Epoch 197/300, Batch

In [9]:
print(np.mean([np.mean(s['test_accuracy'][-10:]) for s in stats_of_runs]))
[np.mean(s['test_accuracy'][-10:]) for s in stats_of_runs]

0.54762


[0.5753999999999999,
 0.5274000000000001,
 0.5264,
 0.5534000000000001,
 0.5679999999999998,
 0.5610000000000002,
 0.5418000000000001,
 0.573,
 0.5262,
 0.5236]

In [ ]:
plt.figure()
stats = stats_of_runs[0]

c_start = stats['centers'][3]
c_end = stats['centers'][-1]

plt.plot(c_start[:,0], c_start[:, 1], 'bo', label='center initialization')
plt.plot(c_end[:,0], c_end[:, 1], 'ro', label='center learned')

all_centers = numpy.stack(stats['centers'], axis=0)
for i in range(all_centers.shape[1]):
    points = all_centers[:,i, :]
    plt.plot(points[:, 0], points[:, 1], '-k')
    
plt.legend()

plt.figure()
plt.plot(stats['train_loss_by_epoch'], label='train_loss')
plt.plot(stats['test_loss_by_epoch'], label='test_loss')
plt.plot(stats['test_accuracy'], label='test_acccuracy')
plt.legend()

plt.figure()
radius = stats['radius']
radius = np.stack(radius, axis=1)
for curve in radius:
    plt.plot(curve)
plt.show()